In [13]:
from collections import deque
from uuid import uuid4
DEFAULT = 0
orcId = -2
danger = DEFAULT
m = DEFAULT
n = DEFAULT
gandalfX = DEFAULT
gandalfY = DEFAULT
gandorX = DEFAULT
gandorY = DEFAULT
ringMembers = []
orcs = []
startFlag = True

In [ ]:
class RingMembers:
    def __init__(self,x,y,id):
        self.x = x
        self.y = y
        self.endX = DEFAULT
        self.endY = DEFAULT
        self.id = id
        self.heu
    
    def setDest(self,x,y):
        self.endX = x
        self.endY = y
        

In [14]:
class Orc:
    def __init__(self,x,y,rank,id):
        self.x = x 
        self.y = y
        self.rank = rank
        self.id = id
    
    def setOrcsInTable(self,table):
        global m,n

        for i in range(m):
            for j in range(n):
                if i == m and j == n:
                    table[i][j].orcIsHere()
                    table[i][j].orcIdSet(self.rank)
        
        for i in range(self.y - self.rank, self.y + self.rank + 1):
            if i in range(0,n):
                temp = self.rank - abs(i - self.y)
                for j in range(-temp, temp + 1):
                    if (self.x + j) in range(0,m): 
                        table[self.x + j][i].rangeRank(self.rank)
                        table[self.x + j][i].orcIdSet(self.id)

        return table

In [15]:
class Node:

    def __init__(self, x, y):

        self.x = x 
        self.y = y
        self.f = 0
        self.g = 0
        self.h = 0
        self.neighbors = []
        self.neighborsDirection = []
        self.previous = None
        self.orc = False
        self.rank = DEFAULT
        self.orcId = DEFAULT

    def orcIsHere(self):
            self.orc = True

    def rangeRank(self,rank):
        self.rank = rank
        
    def orcIdSet(self,id):
        self.orcId = id
    
    def getLoc(self):
        return self.x,self.y

    def addNeighbors(self, table, columns, rows):

        neighbor_x = self.x
        neighbor_y = self.y
    
        if neighbor_x < columns - 1:
            self.neighbors.append(table[neighbor_x+1][neighbor_y])
            self.neighborsDirection.append("D")
        if neighbor_x > 0:
            self.neighbors.append(table[neighbor_x-1][neighbor_y])
            self.neighborsDirection.append("U")
        if neighbor_y < rows -1:
            self.neighbors.append(table[neighbor_x][neighbor_y +1])
            self.neighborsDirection.append("R")
        if neighbor_y > 0: 
            self.neighbors.append(table[neighbor_x][neighbor_y-1])
            self.neighborsDirection.append("L")


In [16]:
class AStar:

    def __init__(self, cols, rows, start, end):

        self.cols = cols
        self.rows = rows
        self.start = start
        self.end = end
        

 
    def cleanOpenSet(openSet, currentNode):

        for i in range(len(openSet)):
            if openSet[i] == currentNode:
                openSet.pop(i)
                break

        return openSet

    def fillTable(table):
        for i in range(m):
            for j in range(n):
                table[i][j] = Node(i,j)
        for i in range(len(orcs)):
            table = orcs[i].setOrcsInTable(table)

        return table
 
    def heu(currentNode, end):

        distance =  abs(currentNode.x - end.x) + abs(currentNode.y - end.y)
        
        return distance

 
    def createTable(cols, rows):

        table = [[0 for i in range(rows)]
                    for j in range(cols)]
        
        return table


    def getNeighbors(table, cols, rows):
        for i in range(cols):
            for j in range(rows):
                table[i][j].addNeighbors(table, cols, rows)
        return table

    def startPath(openSet, closeSet, currentNode, end ,startFlag):
        global danger

        bestWay = 0
        for i in range(len(openSet)):
            if openSet[i].f < openSet[bestWay].f:
                bestWay = i

        currentNode = openSet[bestWay]

        if(startFlag == False):
            if((currentNode.previous.orcId == currentNode.orcId) and (currentNode.rank > 0)):
                danger = danger+1
            elif(currentNode.previous.orcId != currentNode.orcId) and (currentNode.rank > 0):
                danger = 1
            elif(currentNode.previous.orcId != currentNode.orcId) and (currentNode.rank == 0):
                danger =0

        final_path = []
        if currentNode == end:
            temp = currentNode
            while temp.previous:
                final_path.append(temp.previous)
                temp = temp.previous
            
            
        openSet = AStar.cleanOpenSet(openSet, currentNode)
        closeSet.append(currentNode)
        neighbors = currentNode.neighbors
        for neighbor in neighbors:
            if (neighbor in closeSet) or (neighbor.orc == True) or (danger+1 == neighbor.rank):
                continue
            else:
                temp_g = currentNode.g + 1
                control_flag = 0
                for k in range(len(openSet)):
                    if neighbor.x == openSet[k].x and neighbor.y == openSet[k].y:
                        if temp_g < openSet[k].g:
                            openSet[k].g = temp_g
                            openSet[k].h= AStar.heu(openSet[k], end)
                            openSet[k].f = openSet[k].g + openSet[k].h
                            openSet[k].previous = currentNode
                        else:
                            pass
                        control_flag = 1
  
                if control_flag == 1:
                    pass
                else:
                    neighbor.g = temp_g
                    neighbor.h = AStar.heu(neighbor, end)
                    neighbor.f = neighbor.g + neighbor.h
                    neighbor.previous = currentNode
                    openSet.append(neighbor)
        return openSet, closeSet, currentNode, final_path


In [ ]:
def gandalfDistUpdate():
    for i in range(len(ringMembers)):
        start = Node(gandalfX,gandalfY)
        end = Node(ringMembers[i].x,ringMembers.y)
        heus = AStar.heu(start,end)
        ringMembers[i].heu = heus
    ringMembers.sort(key=lambda x:x.heu)

In [17]:
def inputManager():
    global m, n ,ringMembers
    global gandalfX , gandalfY ,gandorX,gandorY,orcs
    m, n = map(int, input("m and n").split())
    gandalfX, gandalfY = map(int, input("start").split())
    gandorX, gandorY = map(int, input("end").split())
    k, l = map(int, input("k and l").split())

    for j in range(k):
        unique_id = str(uuid4())
        x, y, rank = map(int, input().split())
        orcs.append(Orc(x,y,rank,unique_id)) 
    
    for j in range(l):
        unique_id = str(uuid4())
        x, y = map(int, input().split())
        ringMembers.append(RingMembers(x,y,unique_id))

    for j in range(l):
        x, y = map(int, input().split())
        ringMembers[j].setDest(x, y)


    return    

In [18]:
def main():
    global gandalfX, gandalfY
    finished = False
    directions = []    
    inputManager()
    table = AStar.createTable(n, m)
    table = AStar.fillTable(table)
    table = AStar.getNeighbors(table, n, m)
    
    while (len(ringMembers) > 0) and (finished == False):

        startFlag = True
        openSet  = []
        closeSet  = []
        currentNode = None
        final_path  = []
        openSet.append(table[gandalfX][gandalfY])
        if len(ringMembers) == 0:
            end =table[gandorX][gandorY]
        else:
            end = table[ringMembers[0]][ringMembers[0]]

        while len(openSet) > 0:
            openSet, closeSet, currentNode, final_path = AStar.startPath(openSet, closeSet, currentNode,end,startFlag)
            startFlag = False
            if len(final_path) > 0:
                break
        
        final_path.reverse()
        for i in range(len(final_path)):
            temp = final_path[i+1]
            gandalfX = final_path[i].x
            gandalfY = final_path[i].y
            for j in range(len(final_path[i].neighbors)):
                if temp == final_path[i].neighbors[j]:
                    directions.append(final_path[i].neighborsDirection[j])
                    break
        
        gandalfDistUpdate()        

        if (gandalfX == gandorX) and (gandalfY == gandorY):
            finished = True

main()

D
R
R
D
R
D
D
D
D
D
R
R
R
